In [ ]:
import logging
import os

from dotenv import load_dotenv

import openai

from yaaal.core import JSON, Caller, Conversation, Message, PydanticResponseValidatorMixin
from yaaal.core.tools import CallableWithSignature, respond_as_tool
from yaaal.prompts import Extract, ExtractorPrompt, MemoWriterPrompt, SummarizerPrompt, Summary
from yaaal.tools.web import get_page_content
from yaaal.utilities import basic_log_config

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
logger = logging.getLogger(__name__)
basic_log_config(logger)

In [ ]:
current_events = get_page_content("https://en.wikipedia.org/wiki/Portal:Current_events")

In [ ]:
_ = load_dotenv()
if "YAAAL_OPENAI_API_KEY" not in os.environ:
    raise KeyError("YAAAL_OPENAI_API_KEY was not found in environment!")

openai_client = openai.Client(api_key=os.environ["YAAAL_OPENAI_API_KEY"])

In [ ]:
class Summarizer(PydanticResponseValidatorMixin, Caller):
    def __init__(
        self, client, request_params: dict[str, JSON], toolbox: list[Caller | CallableWithSignature] | None = None
    ):
        self.client = client
        self.prompt = SummarizerPrompt
        self.request_params = request_params
        self.toolbox = toolbox
        self.response_validator = Summary

        if self.toolbox:
            self.request_params["tools"] = [openai.pydantic_function_tool(tool) for tool in self.toolbox.values()]

    def __call__(self, *, system_vars: dict, user_vars: dict | None):
        conversation = self.prompt.render(system_vars=system_vars, user_vars=user_vars)
        response = self.client.chat.completions.create(
            messages=conversation.model_dump()["messages"],
            **self.request_params,
        )
        content = response.choices[0].message.content

        try:
            validated = self.validate(response)
        except Exception as e:
            logger.debug(f"Attempting fix for exception raised during validation: {e}")
            repair_messages = self.render_repair(
                response_content=response.choices[0].message.content,
                exception=str(e),
            )
            conversation.messages.extend(repair_messages.messages)

            response = self.client.chat.completions.create(
                messages=conversation.model_dump()["messages"],
                **self.request_params,
            )
            content = response.choices[0].message.content
            return self.validate(content)

        return validated

In [ ]:
summarizer = Summarizer(client=openai_client, request_params={"model": "gpt-4o-mini", "temperature": 0.5})

In [ ]:
summarizer.prompt.render(
    system_vars={"source": current_events}, user_vars={"guidance": "What's happening in the world today?"}
).model_dump()["messages"]

In [ ]:
summary = summarizer(
    system_vars={"source": current_events}, user_vars={"guidance": "What's happening in the world today?"}
)
print(summary.model_dump_json(indent=2))

In [ ]:
# TODO
# simple prompts
# simple Callers
# Caller validations
# Tool Callers
# Nested Tool Callers

In [ ]:
get_page_content.signature().model_json_schema()

In [ ]:
# TODO:
# callers may need context of current conversation to make the correct tool choice or influence the generation
# NOTE: Callers should NOT modify the current conversation!